In [37]:
import twitter_data
import stock_data
from twitter_data import data_creator_predict
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
x = twitter_data.search_predict("Apple Inc")

In [3]:
df = pd.read_csv('training_data.csv')

In [4]:
df.head()

,Unnamed: 0,index,negative,positive,total,Price Change_down,Price Change_up
0,0,0,0.092025,0.343558,163.0,0,1
1,1,1,0.414365,0.287293,181.0,0,1
2,2,2,0.125000,0.615000,200.0,1,0
3,3,3,0.115000,0.245000,200.0,1,0
4,4,4,0.165829,0.261307,199.0,0,1


In [5]:
def xScaler():
    x = np.array([[1.16666667, 0.66666667],[0.6131379 , 1.52368464],[0.9011529 , 1.42572293],[0.91221833, 1.75215763],[0.35060131, 1.75027643],[1.33333333, 0.93333333],[0.        , 2.79100529],[0.3488813 , 1.69110366],[1.0224359 , 3.37703963]])
    
    X_scaler = StandardScaler().fit(x)
    return X_scaler
    

In [59]:
def ml_data_formatter(data5): 
    try:
        data5['Price Change_up'].astype('int64')
    except:
        try:
            data5['Price Change_down'].astype('int64')
            if data5.iloc[0,3] == 1:
                data5.iloc[0,3] = 0
            else:
                data4.iloc[0,3] = 1
        except:
            try:
                data5 = data5.T
                data5['Price Change_up'].astype('int64')
            except:
                try:
                    data5['Price Change_down'].astype('int64')
                    if data5.iloc[0,3] == 1:
                        data5.iloc[0,3] = 0
                    else:
                        data4.iloc[0,3] = 1
                finally:
                    print('Not a valid company')
    data6x = []
    tempx = np.array([])
    for i in range(len(data5)):
        negative = data5.iloc[i,0]
        positive = data5.iloc[i,1]
        if i == 0:
            tempx = np.append(tempx, [negative, positive])
        else:
            tempx[0][0] += negative
            tempx[0][1] += positive
    data6x.append(tempx)
        
    #print(data6x)
    X = np.asarray(data6x)
    #print(X)
    X_scaler = xScaler()
    X_scaled = X_scaler.transform(X)
    
    return X_scaled



In [49]:
def get_sentiment(df):
    negative = []
    positive = []
    for i in range(len(df)):
        negative.append(df.iloc[i,0])
        positive.append(df.iloc[i,1])
    neg_avg = sum(negative)/len(negative)
    pos_avg = sum(positive)/len(positive)
    averages = [neg_avg, pos_avg]
    return averages

In [26]:
def stock_prediction(X, model):
    prediction = model.predict_classes(X)
    if prediction == 1:
        return 'Buy'
    else:
        return 'Sell'

In [27]:
twitter_data = data_creator_predict('Apple Inc')

/Users/PatBeljan/Desktop/Bootcamp/final_project/twitter_data.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  stock['Date'] = stock['Date'].astype('str')


In [28]:
twitter_data

,negative,positive,total,Price Change_up
0,0.16,0.226667,300.0,1


In [10]:
data = pd.DataFrame(df.iloc[:2]).T

In [11]:
data

,0,1
Unnamed: 0,0.000000,1.000000
index,0.000000,1.000000
negative,0.092025,0.414365
positive,0.343558,0.287293
total,163.000000,181.000000
Price Change_down,0.000000,0.000000
Price Change_up,1.000000,1.000000


In [29]:
data2 = ml_data_formatter(twitter_data)

In [35]:
data2[0][1]

-1.9318204314210878

In [31]:
stock_prediction(data2, model)

'Buy'

In [ ]:
X = df[['negative', 'positive', 'total']].values
y = df[['Price Change_down', 'Price Change_up']].values

In [ ]:
X

In [ ]:
X_input = np.asarray([X])

In [15]:
model = load_model("Twitter_Sentiment_Stock_Model.h5")

In [ ]:
X_scaler = StandardScaler().fit(X)
X_train_scaled = X_scaler.transform(X)

In [50]:
def get_Prediction(company_name):
    model = load_model("Twitter_Sentiment_Stock_Model.h5")
    try:
        data = data_creator_predict(company_name)
    except:
        print('Not a valid company name')
        return None
    ml_data = ml_data_formatter(data)
    prediction = stock_prediction(ml_data, model)
    json_return = {}
    json_return['prediction'] = prediction
    averages = get_sentiment(data)
    json_return['positive'] = averages[1]
    json_return['negative'] = averages[0]
    try:
        json_return['Closing Prices'] = stock_data.find_ticker_data(company_name).iloc[-5:,0].tolist()
    except:
        print('Not a valid company name')
        return None
    return json_return

In [41]:
stock_data.find_ticker_data('Apple Inc').iloc[-5:,0].tolist()

[310.33, 316.96, 312.68, 311.34, 314.2]

In [60]:
get_Prediction('Tesla Motors Inc')

/Users/PatBeljan/Desktop/Bootcamp/final_project/twitter_data.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  stock['Date'] = stock['Date'].astype('str')


{'prediction': 'Buy',
 'positive': 0.43,
 'negative': 0.02,
 'Closing Prices': [478.15, 524.86, 537.92, 518.5, 513.95]}